In [1]:
import pandas as pd

In [3]:
basedir = '/scratch/ias41/ae_code'

In [4]:
# MedDRA hierchy
meddra_hier = pd.read_excel(basedir + '/analysis/data/all_faers_and_sider_aes_hier_output.xlsx', skiprows=4)
meddra_hier_selection = meddra_hier.loc[meddra_hier['Primary SOC']=='Y',[' Term','HLT','SOC','PT']].drop_duplicates()
meddra_hier_selection['HLT'] = meddra_hier_selection['HLT'].apply(lambda x: x.upper())

# Previously reported associations
# Known associations, merge with known hierarchy HLT
known_associations = pd.read_excel(basedir + '/prev_reported_safety_associations/data/safety_meddra_annotated_effects.xls')
known_associations['Annotated MedDRA PT'] = known_associations['Annotated MedDRA PT'].apply(lambda x: x.upper())
known_meddra_hier = pd.read_excel(basedir + '/prev_reported_safety_associations/data/safety_meddra_annotated_effects_for_hierarchy_output.xlsx', skiprows=4)
known_meddra_hier['PT'] = known_meddra_hier['PT'].apply(lambda x: x.upper())
known_meddra_hier[' Term'] = known_meddra_hier[' Term'].apply(lambda x: x.upper())
known_meddra_hier['HLT'] = known_meddra_hier['HLT'].apply(lambda x: x.upper())
known_meddra_hier_selection = known_meddra_hier.loc[known_meddra_hier['Primary SOC']=='Y',['PT','HLT',' Term']].drop_duplicates()
known_merged = known_associations.merge(known_meddra_hier_selection, left_on='Annotated MedDRA PT', right_on=' Term')

hlt_manual = pd.read_excel(basedir + '/prev_reported_safety_associations/data/safety_meddra_manually_annotated_hlt_effects.xls', index=False)
hlt_manual.rename(columns={'Annotated MedDRA HLT': 'HLT'}, inplace=True)
hlt_manual['HLT'] = hlt_manual['HLT'].apply(lambda x: x.upper())
hlt_manual.drop(columns=['Annotated MedDRA HLT Code'])

known_merged = pd.concat([known_merged, hlt_manual], sort=False).reset_index(drop=True)

known_tuples = set([(x[1]['Accession'], x[1]['HLT']) for x in known_merged.iterrows()])

In [10]:
known_merged['Reference'].drop_duplicates()

0     Bowes J, Brown AJ, Hamon J, Jarolimek W, Sridh...
1     Lynch JJ 3rd, Van Vleet TR, Mittelstadt SW, Bl...
35    Whitebread S, Hamon J, Bojanic D, Urban L. Key...
Name: Reference, dtype: object

In [9]:
known_merged.groupby('Reference')['Accession'].nunique()

Reference
Bowes J, Brown AJ, Hamon J, Jarolimek W, Sridhar A, Waldron G, Whitebread S. Reducing safety-related drug attrition: the use of in vitro pharmacological profiling. Nat Rev Drug Discov. 2012 Dec;11(12):909-22.    45
Lynch JJ 3rd, Van Vleet TR, Mittelstadt SW, Blomme EAG. Potential functional and pathological side effects related to off-target pharmacological activity. J Pharmacol Toxicol Methods. 2017 Sep;87:108-126.        78
Whitebread S, Hamon J, Bojanic D, Urban L. Keynote review: in vitro safety pharmacology profiling: an essential tool for successful drug development. Drug Discov Today. 2005 Nov 1;10(21):1421-33.                 34
Name: Accession, dtype: int64

In [20]:
bowes_targets = set(known_merged.loc[known_merged['Reference'].str.contains('Bowes J'),'Accession'].drop_duplicates())
whitebread_targets = set(known_merged.loc[known_merged['Reference'].str.contains('Keynote'),'Accession'].drop_duplicates())
lynch_targets = set(known_merged.loc[known_merged['Reference'].str.contains('pathological'),'Accession'].drop_duplicates())

In [22]:
len(lynch_targets & bowes_targets)

38

In [23]:
len(bowes_targets)

45

In [24]:
38/45

0.8444444444444444

In [28]:
38/len(lynch_targets)

0.48717948717948717

In [26]:
len(whitebread_targets & bowes_targets)

16

In [27]:
16/45

0.35555555555555557

In [30]:
len(lynch_targets & whitebread_targets)

26

In [31]:
26/34

0.7647058823529411

In [32]:
26/len(lynch_targets)

0.3333333333333333